In [1]:
from operator import add, mul
from functools import reduce, partial
import math
from tqdm import tqdm

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

https://adventofcode.com/2024/day/7

Notes:
- Branch-pruning idea taken from reddit, other optimisations definitely doable
- Working backwards from target with - and / would be much quicker, since you can prune branches much earlier
- Set operations are surprisingly expensive. Using lists instead is quicker for this solution (there likely aren't many duplicates)

In [3]:
## Part 1
# Equation is true if the values can be combined with + or * ops to form the total
# Ignore precedence rules! Evaluate left-to-right

# Brute-force option:
#  Try every op in every position. 2 choices at each step -> n^2 total combinations
#  Minor possible improvement: We can use a set to skip identical totals
#   ^ In practice, using lists instead of sets speeds up the result from ~80it/s -> ~140it/s!
#  May be worth trying out numpy for quick array ops

In [4]:
# func that works with reduce(f, it, initial)
def apply_ops(current_totals: list[int], y, *, ops=(add, mul), target=None) -> list[int]:
    # Roughly 2x speedup on Part 2 if pruning results > target!
    new_totals = [op(x, y) for x in current_totals for op in ops]
    if target is not None:
        new_totals = [n for n in new_totals if n <= target]
    return new_totals

def get_calibration_result(lines: list[str], ops) -> int:
    calibration_result = 0
    for line in tqdm(lines):
        target, values = line.split(": ")
        target = int(target)
        values = map(int, values.split(" "))
        possible_results = reduce(partial(apply_ops, target=target, ops=ops), values, [next(values)])
        # print(possible_results)
        if target in possible_results:
            # print(f"{target} found!")
            calibration_result += target
    return calibration_result


In [5]:
get_calibration_result(lines, (add, mul))

100%|██████████| 9/9 [00:00<00:00, 37338.02it/s]


3749

In [6]:
## Part 2
# New operator type! Concat ||
# We could be smart by implementing this as concat(a,b) = a * (10 ^ b digits) + b
# But string concat is quicker to implement and less error-prone
def concat(a: int, b: int) -> int:
    return int(str(a) + str(b))

# def concat(a: int, b: int) -> int:
#     b_digits = int(math.log10(b)) + 1
#     return a * (10 ** b_digits) + b

get_calibration_result(lines, (add, mul, concat))

100%|██████████| 9/9 [00:00<00:00, 15382.53it/s]


11387